In [22]:
!pip install -qU llama-index yfinance newsapi-python langchain-groq llama-index-llms-langchain

In [24]:
import os
import getpass
import yfinance as yf
from newsapi import NewsApiClient
from llama_index.core.tools import FunctionTool
from langchain_groq import ChatGroq
from llama_index.core.agent import ReActAgent
from llama_index.llms.langchain import LangChainLLM

In [28]:
!pip install langchain_community -q

# **SETUP: Use Groq and NewsAPI keys**

In [19]:
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")
os.environ["NEWS_API_KEY"] = getpass.getpass("Enter your NewsAPI.org API Key: ")

Enter your Groq API Key: ··········
Enter your NewsAPI.org API Key: ··········


# **1. DEFINE CUSTOM TOOLS**

In [20]:
def get_stock_price(ticker_symbol: str) -> str:
    """
    Use this tool to get the current stock price for a given ticker symbol.

    Args:
        ticker_symbol (str): The stock ticker symbol (e.g., 'AAPL' for Apple).
    """
    try:
        stock = yf.Ticker(ticker_symbol)
        price = stock.history(period="1d")['Close'].iloc[-1]
        return f"The current stock price of {ticker_symbol} is ${price:.2f}"
    except Exception as e:
        return f"Could not find stock price for ticker '{ticker_symbol}'. Error: {e}"

def get_company_news(company_name: str) -> str:
    """
    Use this tool to get the latest news articles for a given company name.

    Args:
        company_name (str): The name of the company (e.g., 'Apple', 'Microsoft').
    """
    try:
        newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])
        top_headlines = newsapi.get_everything(
            q=company_name,
            language='en',
            sort_by='publishedAt',
            page_size=3
        )
        articles = top_headlines.get('articles', [])
        if not articles:
            return f"No recent news found for {company_name}."

        news_summary = f"Here are the top 3 recent news headlines for {company_name}:\n"
        for article in articles:
            news_summary += f"- Title: {article['title']}\n  Source: {article['source']['name']}\n"
        return news_summary
    except Exception as e:
        return f"Could not fetch news for '{company_name}'. Error: {e}"

def get_company_info(ticker_symbol: str) -> str:
    """
    Use this tool to get a summary of a company's business activities.

    Args:
        ticker_symbol (str): The stock ticker symbol (e.g., 'AAPL' for Apple).
    """
    try:
        company = yf.Ticker(ticker_symbol)
        return company.info.get('longBusinessSummary', f"No summary available for {ticker_symbol}.")
    except Exception as e:
        return f"Could not fetch info for ticker '{ticker_symbol}'. Error: {e}"

print("All custom tools for the stock market agent have been defined.")

All custom tools for the stock market agent have been defined.


# **2. CREATE THE AGENT WITH THE FREE LLM**

In [32]:
# Create a list of tools
tool_list = [
    FunctionTool.from_defaults(fn=get_stock_price),
    FunctionTool.from_defaults(fn=get_company_news),
    FunctionTool.from_defaults(fn=get_company_info),
]

# Initialize the free LLM from Groq
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

# Wrap the LangChain model for LlamaIndex compatibility using the correct import
lm = LangChainLLM(llm)

# Create the agent using the compatible ReActAgent
agent = ReActAgent.from_tools(
    tools=tool_list,
    llm=lm,
    verbose=True
)

print("\nStock Market Agent created successfully.")


Stock Market Agent created successfully.


# **3. CHAT WITH THE AGENT**

In [33]:
# --- Example 1: Trigger the get_stock_price tool ---
print("\n--- TEST 1: Getting a stock price ---")
response1 = agent.chat("What is the current stock price for NVIDIA (NVDA)?")
print(f"\nAgent Response: {response1}")

# --- Example 2: Trigger the get_company_news tool ---
print("\n--- TEST 2: Getting company news ---")
response2 = agent.chat("What's the latest news on Tesla?")
print(f"\nAgent Response: {response2}")

# --- Example 3: A complex query requiring multiple tool calls ---
print("\n--- TEST 3: Complex multi-tool query ---")
response3 = agent.chat("Give me the current price and recent news for Google (GOOGL).")
print(f"\nAgent Response: {response3}")


--- TEST 1: Getting a stock price ---
> Running step 80b1a1db-35a1-4aa7-a774-15442a121819. Step input: What is the current stock price for NVIDIA (NVDA)?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_stock_price
Action Input: {'ticker_symbol': 'NVDA'}
Observation: The current stock price of NVDA is $143.96
> Running step 9cece0f0-2b01-44c5-acc7-51d2416db922. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The current stock price for NVIDIA (NVDA) is $143.96.

Agent Response: The current stock price for NVIDIA (NVDA) is $143.96.

--- TEST 2: Getting company news ---
> Running step 077982c4-4286-4183-88be-53833e4e5b5b. Step input: What's the latest news on Tesla?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_company_news
Action Input: {'company_name': 'Tesla'}
Observation: H

ERROR:yfinance:HTTP Error 401: 


Thought: I also need to get a summary of Google's business activities.
Action: get_company_info
Action Input: {'ticker_symbol': 'GOOGL'}
Observation: No summary available for GOOGL.
> Running step e6acf1f4-ab35-451d-9ac5-795f402f5eeb. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The current stock price of Google (GOOGL) is $168.94, and the recent news headlines include articles about watching Trinidad and Tobago vs Haiti in the USA, a password leak affecting major tech companies, and a triumph for AEW and Pro Wrestling in Mexico.

Agent Response: The current stock price of Google (GOOGL) is $168.94, and the recent news headlines include articles about watching Trinidad and Tobago vs Haiti in the USA, a password leak affecting major tech companies, and a triumph for AEW and Pro Wrestling in Mexico.
